# Passo-a-passo do PCA

1. Normalizar os dados
2. Cálculo da matriz de covariância entre os atributos
3. Autodecomposição da matriz de covariância - decompor a matriz de covariância em autovalores e autovetores
4. Seleção dos autovalores e autovetores mais importantes - ordenar os autovetores com base nos autovalores e selecionar os k primeiros
5. Cálculo da projeção dos autovetores selecionados para obtenção da matriz de projeção.

In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [122]:
df = pd.read_csv('data/dermatology.data')

In [123]:
df.columns = np.arange(0, 35)

In [124]:
# aplicando pré-processamento à coluna 33, que está em formato de string,
# com alguns valoroes inconsistentes, transformando-os em inteiros
df[33] = df[33].apply(lambda x: x if x != '?' else '0').apply(int)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       365 non-null    int64
 1   1       365 non-null    int64
 2   2       365 non-null    int64
 3   3       365 non-null    int64
 4   4       365 non-null    int64
 5   5       365 non-null    int64
 6   6       365 non-null    int64
 7   7       365 non-null    int64
 8   8       365 non-null    int64
 9   9       365 non-null    int64
 10  10      365 non-null    int64
 11  11      365 non-null    int64
 12  12      365 non-null    int64
 13  13      365 non-null    int64
 14  14      365 non-null    int64
 15  15      365 non-null    int64
 16  16      365 non-null    int64
 17  17      365 non-null    int64
 18  18      365 non-null    int64
 19  19      365 non-null    int64
 20  20      365 non-null    int64
 21  21      365 non-null    int64
 22  22      365 non-null    int64
 23  23      365 non

In [126]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,3,3,3,2,1,0,0,0,1,1,1,0,0,1,0,1,2,0,2,2,2,2,2,1,0,0,0,0,0,0,0,1,0,8,1
1,2,1,2,3,1,3,0,3,0,0,0,1,0,0,0,1,2,0,2,0,0,0,0,0,2,0,2,3,2,0,0,2,3,26,3
2,2,2,2,0,0,0,0,0,3,2,0,0,0,3,0,0,2,0,3,2,2,2,2,0,0,3,0,0,0,0,0,3,0,40,1
3,2,3,2,2,2,2,0,2,0,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,2,2,3,2,3,0,0,2,3,45,3
4,2,3,2,0,0,0,0,0,0,0,0,0,2,1,0,2,2,0,2,0,0,0,1,0,0,0,0,2,0,0,0,1,0,41,2


In [127]:
X = df.drop(10, axis=1).values
y = df[10].values

In [128]:
X.shape, X

((365, 34), array([[ 3,  3,  3, ...,  0,  8,  1],
        [ 2,  1,  2, ...,  3, 26,  3],
        [ 2,  2,  2, ...,  0, 40,  1],
        ...,
        [ 3,  2,  2, ...,  3, 28,  3],
        [ 2,  1,  3, ...,  3, 50,  3],
        [ 3,  2,  2, ...,  0, 35,  1]]))

In [129]:
y.shape, y

((365,),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 

Normalmente, aqui seria o momento de aplicar uma normalização aos dados, para usá-los futuramente no PCA.
No entanto, como estamos lidando majoritariamente com valores categóricos e em faixas parecidas de valores, deixaremos o dataset como está.

Além disso, utilizaremos como alvo de classificação a coluna de family history, identificada como coluna 10 no dataframe.

## Questão 1: aplicação do PCA para diminuir dimensão.

### 1.1. Determinar uma dimensão adequada que conserve 95% da variância dos dados originais.

In [130]:
# utilizando PCA de 2 a 34 componentes
n_values = np.arange(2, 35)
# inicializando lista de dataframes resultantes
# e lista de variâncias para cada resultado
dfs = []
explained_variances = {}
for n in n_values:
  # instanciando método para n componentes
  pca = PCA(n_components=n)
  # extraindo n principais componentes
  principal_components = pca.fit_transform(X)
  # calculando variância
  explained_variances[n] = pca.explained_variance_ratio_
  # salvando em um dataframe
  cols = []
  for i in range(n):
    cols.append('PC{}'.format(i))
  df_pc = pd.DataFrame(data = principal_components, columns = cols)
  dfs.append(df_pc)

In [131]:
# determinando qual número de componentes possui 95% de variância no total
print('Dimensão/Variância\n')
for i in n_values:
  print((i, explained_variances[i].sum()))

Dimensão/Variância

(2, 0.9390804307521952)
(3, 0.9569705174993853)
(4, 0.9650853873823323)
(5, 0.9700812981542659)
(6, 0.9732888569443068)
(7, 0.9761618682428007)
(8, 0.9788701394272223)
(9, 0.9809940046940616)
(10, 0.982997079750268)
(11, 0.9848255975511261)
(12, 0.9865640302795203)
(13, 0.9880941488132752)
(14, 0.9894806959615435)
(15, 0.9907357501379757)
(16, 0.9918064758496246)
(17, 0.9927315744153684)
(18, 0.9936332961973324)
(19, 0.9944745435594977)
(20, 0.9952527598651197)
(21, 0.9959786263907383)
(22, 0.996568040200909)
(23, 0.9971023839012103)
(24, 0.9974933319345243)
(25, 0.9978710567701563)
(26, 0.9982401396545425)
(27, 0.9985723118625662)
(28, 0.9988933220380471)
(29, 0.999160579045329)
(30, 0.9994035873964853)
(31, 0.9996339040022576)
(32, 0.9998186911466042)
(33, 0.9999546832819283)
(34, 1.0000000000000004)


Analisando os resultados acima, podemos concluir que, utilizando 3 componentes, ou seja, reduzindo os dados a 3 features, alcançamos uma variância de 95% dos dados originais.

In [82]:
print('Variância utilizando 3 dimensões: ', str(100*round(explained_variances[3].sum(), 3)), '%')

Variância utilizando 3 dimensões:  95.7 %


### 1.2. Determinar uma dimensão adequada que maximize a taxa de acerto para o novo conjunto de vetores.

Utilizaremos, aqui, o método de Regressão Logística para classificar a coluna y definida anteriormente. Outros métodos poderiam ser utilizados, mas, a nível de exemplo, foi escolhido este pela simiplicidade e fácil aplicação.

Além disso, utilizaremos os novos dados (lista dfs), reduzidos em dimensão, resultados do item anterior. Nele, temos os dados originais reduzidos a n dimensões, com n variando de 2 a 34, junto com o vetor y definido anteriormente.

Finalmente, para comparação de performance de classificação, utilizaremos a acurácia do modelo, dado pela razão:

                        acurácia = classificações corretas / quantidade total de pontos

In [132]:
# treinando regressão logística para cada conjunto de dados
# reduzidos em dimensão obtidos no item anterior
print('Dimensão/Acurácia\n')
for df_new, n in zip(dfs, n_values):
  X_new = df_new.copy()
  # dividindo dados de treino e teste, guardando 30% dos dados para teste
  X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.3, random_state = 42)
  # instanciando e treinando regressão logística
  lr = LogisticRegression()
  lr.fit(X_train, y_train)
  # obtendo performance do modelo (acurácia)
  classification_score = lr.score(X_test, y_test)
  print(str(n), ': ', classification_score)

Dimensão/Acurácia

2 :  0.8727272727272727
3 :  0.8727272727272727
4 :  0.8727272727272727
5 :  0.8818181818181818
6 :  0.8727272727272727
7 :  0.8727272727272727
8 :  0.8727272727272727
9 :  0.8727272727272727
10 :  0.8727272727272727
11 :  0.8727272727272727
12 :  0.8727272727272727
13 :  0.8727272727272727
14 :  0.8727272727272727
15 :  0.8727272727272727
16 :  0.8727272727272727
17 :  0.8727272727272727
18 :  0.8727272727272727
19 :  0.8727272727272727
20 :  0.8727272727272727
21 :  0.8727272727272727
22 :  0.8636363636363636
23 :  0.8636363636363636
24 :  0.8545454545454545
25 :  0.8545454545454545
26 :  0.8545454545454545
27 :  0.8545454545454545
28 :  0.8545454545454545
29 :  0.8545454545454545
30 :  0.8545454545454545
31 :  0.8545454545454545
32 :  0.8545454545454545
33 :  0.8545454545454545
34 :  0.8545454545454545


Pelos resultados obtidos acima, podemos concluir que o número de dimensões ao qual reduzir os dados originais que maximiza a taxa de acerto é 5.

Obtemos, portanto, um número diferente do obtido no item anterior.

Dessa forma, concluímos que, para obter 95% da variância dos dados originais, devemos reduzi-los a 3 dimensões. No entanto, para maximizar a performance do classificador, devemos reduzir os dados a 5 dimensões.

## Questão 2: aplicar PCA para selecionar os atributos mais importantes/relevantes do conjunto de dados original. Determinar a taxa média de acerto e seu desvio-padrão antes e após selecionar os atributos mais relevantes.

In [133]:
model = RandomForestClassifier()
model.fit(X, y)
importance = model.feature_importances_
# organizando atributos por importância
importances = {}
for i,v in enumerate(importance):
	# print('Atributo: %0d, Importância: %.5f' % (i,v))
  importances[i] = v

In [142]:
importances_df = pd.DataFrame(importance.transpose(), columns=[0], index=df.drop(10, axis=1).columns)

In [147]:
print('Importância/relevância ordenada por atributo')
importances_df.sort_values(0, ascending=False)

Importância/relevância ordenada por atributo


,0
33,0.119148
8,0.086902
19,0.053902
9,0.045775
21,0.044874
23,0.044331
31,0.043079
3,0.039573
25,0.039314
13,0.037811
